# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [51]:
# Do all imports and installs here
import pandas as pd
import pyspark.sql.functions as f
from IPython.display import display, HTML


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

In this project we will be building analytics database using [Yelp dataset](https://www.yelp.com/dataset). This analytics table can be used to answer questions like: 
- How different discount, customer relationship programs, etc on reviews. So basically data analyst should be able to run a query and see how the reviews score changes during program time vs other times
- Have ability to fitler out negative(less than average review score for given business)reviews and find top used words/phrases
- Find top users that provided most value to the business using reviews/tips

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

Dataset contains a number of of newline delimeted json files.

#### Dataset description 
From [the dataset description](https://www.yelp.com/dataset/documentation/main)

Each file is composed of a single object type, one JSON-object per-line.

Take a look at some examples to get you started: https://github.com/Yelp/dataset-examples.

Note: the follow examples contain inline comments, which are technically not valid JSON. This is done here to simplify the documentation and explaining the structure, the JSON files you download will not contain any comments and will be fully valid JSON.

Sources:

- business.json - Contains business data including location data, attributes, and categories.

```json 
{
    // string, 22 character unique string business id
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // string, the business's name
    "name": "Garaje",

    // string, the full address of the business
    "address": "475 3rd St",

    // string, the city
    "city": "San Francisco",

    // string, 2 character state code, if applicable
    "state": "CA",

    // string, the postal code
    "postal code": "94107",

    // float, latitude
    "latitude": 37.7817529521,

    // float, longitude
    "longitude": -122.39612197,

    // float, star rating, rounded to half-stars
    "stars": 4.5,

    // integer, number of reviews
    "review_count": 1198,

    //TODO: convert to boolean

    // integer, 0 or 1 for closed or open, respectively
    "is_open": 1,

    //TODO: do we need this?
    
    // object, business attributes to values. note: some attribute values might be objects
    "attributes": {
        "RestaurantsTakeOut": true,
        "BusinessParking": {
            "garage": false,
            "street": true,
            "validated": false,
            "lot": false,
            "valet": false
        },
    },

    // an array of strings of business categories
    "categories": [
        "Mexican",
        "Burgers",
        "Gastropubs"
    ],

    // an object of key day to value hours, hours are using a 24hr clock
    "hours": {
        "Monday": "10:00-21:00",
        "Tuesday": "10:00-21:00",
        "Friday": "10:00-21:00",
        "Wednesday": "10:00-21:00",
        "Thursday": "10:00-21:00",
        "Sunday": "11:00-18:00",
        "Saturday": "10:00-21:00"
    }
}
```
- review.json - Contains full review text data including the user_id that wrote the review and the business_id the review is written for.
```json
{
    // string, 22 character unique review id
    "review_id": "zdSx_SD6obEhz9VrW9uAWA",

    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "Ha3iJu77CxlrFm-vQRs_8g",

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // integer, star rating
    "stars": 4,

    // string, date formatted YYYY-MM-DD
    "date": "2016-03-09",

    // string, the review itself
    "text": "Great place to hang out after work: the prices are decent, and the ambience is fun. It's a bit loud, but very lively. The staff is friendly, and the food is good. They have a good selection of drinks.",

    //TODO: check min max for next values
    
    // integer, number of useful votes received
    "useful": 0,

    // integer, number of funny votes received
    "funny": 0,

    // integer, number of cool votes received
    "cool": 0
}
```
- user.json - User data including the user's friend mapping and all the metadata associated with the user.
```json
{
    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "Ha3iJu77CxlrFm-vQRs_8g",

    // string, the user's first name
    "name": "Sebastien",

    // integer, the number of reviews they've written
    "review_count": 56,

    // string, when the user joined Yelp, formatted like YYYY-MM-DD
    "yelping_since": "2011-01-01",

    // array of strings, an array of the user's friend as user_ids
    "friends": [
        "wqoXYLWmpkEH0YvTmHBsJQ",
        "KUXLLiJGrjtSsapmxmpvTA",
        "6e9rJKQC3n0RSKyHLViL-Q"
    ],

    // integer, number of useful votes sent by the user
    "useful": 21,

    // integer, number of funny votes sent by the user
    "funny": 88,

    // integer, number of cool votes sent by the user
    "cool": 15,

    // integer, number of fans the user has
    "fans": 1032,

    // array of integers, the years the user was elite
    "elite": [
        2012,
        2013
    ],

    //TODO: do we need this?
    
    // float, average rating of all reviews
    "average_stars": 4.31,

    // integer, number of hot compliments received by the user
    "compliment_hot": 339,

    // integer, number of more compliments received by the user
    "compliment_more": 668,

    // integer, number of profile compliments received by the user
    "compliment_profile": 42,

    // integer, number of cute compliments received by the user
    "compliment_cute": 62,

    // integer, number of list compliments received by the user
    "compliment_list": 37,

    // integer, number of note compliments received by the user
    "compliment_note": 356,

    // integer, number of plain compliments received by the user
    "compliment_plain": 68,

    // integer, number of cool compliments received by the user
    "compliment_cool": 91,

    // integer, number of funny compliments received by the user
    "compliment_funny": 99,

    // integer, number of writer compliments received by the user
    "compliment_writer": 95,

    // integer, number of photo compliments received by the user
    "compliment_photos": 50
}
```

- checkin.json - Checkins on a business.
```json
{
    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg"

    // string which is a comma-separated list of timestamps for each checkin, each with format YYYY-MM-DD HH:MM:SS
    "date": "2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02"
}
```
- tip.json - Tips written by a user on a business. Tips are shorter than reviews and tend to convey quick suggestions.
```json
{
    // string, text of the tip
    "text": "Secret menu - fried chicken sando is da bombbbbbb Their zapatos are good too.",

    // string, when the tip was written, formatted like YYYY-MM-DD
    "date": "2013-09-20",

    // integer, how many compliments it has
    "compliment_count": 172,

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "49JhAJh8vSQ-vM4Aourl0g"
}
```

In [2]:
# Read in the data here

In [2]:
from pyspark.sql import SparkSession

# config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
%time
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

prev = spark.conf.get("spark.sql.execution.arrow.enabled")  # Keep its default value.
ps.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.87 µs


21/10/27 01:08:30 WARN Utils: Your hostname, lwo1-lhp-a05809 resolves to a loopback address: 127.0.1.1; using 192.168.0.201 instead (on interface wlp0s20f3)
21/10/27 01:08:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/27 01:08:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


NameError: name 'ps' is not defined

In [ ]:
uniq(df):
    for c in df.columns
        

In [90]:
def info(df):
    display(df.summary().toPandas())
    print("Unique values:")
    k=[c:df.select(c).distinct().count() for c in df.columns]
        
#     print("Nan values:")
#     df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in df.columns]).toPandas()
#     print("Null values:")
#     df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in 
#            df.columns]).toPandas().T

SyntaxError: invalid syntax (1052124531.py, line 4)

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.


#### Businesses

In [86]:
%%time
df=spark.read.json("./data/yelp_academic_dataset_business.json")
df.cache()
info(df)
# kdf=df.toPandas()

21/10/27 01:46:48 WARN CacheManager: Asked to cache already cached data.        


,summary,address,business_id,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,count,160585,160585,160470,160585,160585,160585,160585,160585,160585,160585,160585,160585
1,mean,943.6,None,None,None,0.7674938506087119,38.75979407442441,-94.26621179891377,895.5384615384615,47291.79816244746,51.96454837002211,3.6569542609832797,None
2,stddev,739.4905678911666,None,None,None,0.42243123833886226,7.138042289860178,19.9754464384025,853.6057457812532,35625.44084124275,130.0304475755726,0.943604488456221,None
3,min,,--0DF12EMHYI8XIgoFha6A,"3D Printing, Graphic Design, Local Services, P...",51 Richard Beall Hwy 17-92,0,27.998972,-123.3939285,\t EXPRESS CARE KYLE,,5,1.0,ABE
4,25%,460.0,None,None,None,1,30.3558863,-122.5895882649,76.0,2493.0,8,3.0,None
5,50%,900.0,None,None,None,1,42.1772198,-84.3833806851,529.0,32922.0,17,4.0,None
6,75%,1201.0,None,None,None,1,45.458171,-81.2887959927,1630.0,78756.0,44,4.5,None
7,max,​5753 Sydney Ann Ct,zzzKmD9Mj6WtJwJUhA_1dg,"Zoos, Parks, Active Life, Local Flavor",watertown,1,49.49,71.113271,中餐廳 The Real Chinese Restaurant,V8T 1P5,9185,5.0,WY


Unique values:


Py4JError: org.apache.spark.sql.functions.sum_distinct does not exist in the JVM

In [10]:
kdf.shape

(160585, 14)

In [11]:
print("Missing values")
c=kdf.isnull().sum()
print(c[c>0])

Missing values
attributes    14992
categories      115
hours         27341
dtype: int64


In [19]:
df.select([col[0] for col in df.dtypes if col[1] != 'string']).describe().toPandas()

,summary,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,fans,funny,review_count,useful
0,count,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457,2189457
1,mean,3.6538161105699274,2.5026232531627706,0.13045700372284086,2.5026232531627706,1.633913340156943,0.06127409672809286,0.2736518689337128,1.2354332603928737,0.9881007025942962,2.6764133755538473,0.1628577313918474,0.9526750239899665,20.473540699817352,1.3792186829885218,15.39467959407287,21.697721398502004,38.056672955897284
2,stddev,1.1538609330757066,83.63695759997829,10.767452899622612,83.63695759997829,64.40826658665287,9.473195831748187,11.998873844683619,39.82064066188779,87.43188645190922,114.14251460043835,14.187798789819569,29.47058162153114,466.82963889748737,16.866749723244446,353.269747281276,76.01254770183907,535.2625345401981
3,min,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,max,5.0,46858,13654,46858,25304,12669,13501,38322,82630,90858,14180,15446,198451,12116,172041,15686,204380


In [13]:
kdf.describe(include="all")

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
count,160585,145593,160585,160470,160585,133244,160585.000000,160585.000000,160585.000000,160585,160585,160585.000000,160585.000000,160585
unique,123895,67907,160585,88115,836,50857,NaN,NaN,NaN,125850,5779,NaN,NaN,31
top,,"(None, None, None, None, None, None, None, Non...",6iYb2HFDywm3zjuRg0shjw,"Beauty & Spas, Hair Salons",Austin,"(0:0-0:0, 0:0-0:0, 0:0-0:0, 0:0-0:0, 0:0-0:0, ...",NaN,NaN,NaN,Starbucks,78704,NaN,NaN,MA
freq,6726,9316,1,757,22416,5708,NaN,NaN,NaN,852,2084,NaN,NaN,36012
mean,NaN,NaN,NaN,NaN,NaN,NaN,0.767494,38.759794,-94.266212,NaN,NaN,51.964548,3.656954,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,0.422431,7.138042,19.975446,NaN,NaN,130.030448,0.943604,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,27.998972,-123.393929,NaN,NaN,5.000000,1.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,30.355886,-122.589583,NaN,NaN,8.000000,3.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,42.177366,-84.383281,NaN,NaN,17.000000,4.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,45.458531,-81.288501,NaN,NaN,44.000000,4.500000,NaN


In [15]:
df['city'].duplicated().sum()

TypeError: 'Column' object is not callable

#### Reviews


In [16]:
df=spark.read.json("./data/yelp_academic_dataset_review.json")
df.createOrReplaceTempView("reviews")

In [ ]:
# https://stackoverflow.com/questions/44413132/count-the-number-of-missing-values-in-a-dataframe-spark
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()


#### Count empty values 

In [27]:
#https://towardsdatascience.com/data-prep-with-spark-dataframes-3629478a1041
df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in df.columns]).toPandas()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,215,0,0,0,0


#### Count nulls

In [35]:
df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in 
           df.columns]).toPandas().T

,0
average_stars,0
compliment_cool,0
compliment_cute,0
compliment_funny,0
compliment_hot,0
compliment_list,0
compliment_more,0
compliment_note,0
compliment_photos,0
compliment_plain,0


In [31]:
df.filter(f.isnan("name")).toPandas()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,3.89,1,0,1,1,1,1,1,0,2,...,77,,9,"-z2L3_8QgJv2qE1e5Y7N4g, 5swgdKYbC0soxYqRF97IbQ...",118,Nan,265,306,-0DGavcoDjwNU70HGcfRWA,2008-05-28 17:13:35
1,3.64,2,0,2,2,0,1,1,0,5,...,67,"2017,2018,2019,20,20",12,"wbGx1S9-MUOhoR0pcsziRQ, e6UL6lDHAw9P06k-EXqj_w...",79,Nan,165,212,o8klaDarNBztToVrh6haDg,2007-04-05 17:35:05
2,4.67,0,0,0,0,0,0,0,0,0,...,2,,0,"b6-akepsg0N12HHDRKg17w, E71PuojehaxhzawTT0Y4EA",3,Nan,3,9,wHrcGE3baORRhuXxdK1vAQ,2010-03-10 21:02:21
3,4.35,0,0,0,1,0,0,0,0,1,...,9,,1,"kjBI_bok4SEMACuP3T-HoQ, q07LeULmjEXFJTz3oq5T6w...",11,Nan,61,23,14nzk-Yf6eVDdhB5A43c8g,2011-06-25 22:11:30
4,3.68,0,0,0,0,2,0,0,1,1,...,31,,1,"uO_awUl_dB99iBgT0m7WDg, xbfFwxIVFLSKAgDZ-JIIuQ...",19,Nan,60,133,kGGfK4vXzZwS1PREcSC0fw,2010-05-23 03:58:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,5.00,0,0,0,0,0,0,0,0,0,...,1,,0,None,0,Nan,2,1,YU-7vELmIM6P4phJh9nuaw,2016-07-10 10:37:00
211,5.00,0,0,0,0,0,0,0,0,0,...,0,,0,None,0,Nan,1,0,PyX-uS9igqO-or60hgM1wA,2012-12-29 16:57:46
212,2.57,0,0,0,0,0,0,0,0,0,...,1,,0,None,0,Nan,6,7,tOVJ4FCppjio-cKHDKlGNQ,2015-08-25 01:04:05
213,3.40,0,0,0,0,0,0,0,0,0,...,0,,0,None,1,Nan,5,10,4m33jLU-JMfRiyav_wiTdA,2014-10-22 19:58:01


In [ ]:
"{:,}".format(df.count())

In [ ]:
spark.sql("SELECT * FROM reviews WHERE text='' LIMIT 10").toPandas()

In [ ]:
df.limit(5).toPandas().head()

#### Users

In [17]:
%%time
df=spark.read.json("./data/yelp_academic_dataset_user.json")

CPU times: user 26.9 ms, sys: 4.29 ms, total: 31.2 ms
Wall time: 10.9 s


In [ ]:
#https://stackoverflow.com/questions/3154460/python-human-readable-large-numbers

"{:,}".format(df.count())

In [ ]:
df.summary().toPandas()

In [ ]:
df.limit(5).toPandas().head()

#### Cleaning Steps
Document steps necessary to clean the data

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [32]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [33]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.